<a href="https://colab.research.google.com/github/grommitt/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example


# load iris and digits datasets
# the task of the image dataset is to look at the image and predict the digit
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()

In [0]:
# this is the dataset with all of the numbers with the corresponding images that are learned
digits.target

In [0]:

# the sklearn.svm.SVC implements support vector classification
# the estimator's constructor uses the model's parameters for instructions
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
# fit classifier estimator instance to the model (so it can learn from it)
# do this by pass training set to the fit method

# use all the images from digits dataset, except for the last one so there is one image left to predict
# this is done by setting the digits.data parameter to [:-1]
clf.fit(digits.data[:-1], digits.target[:-1])

In [0]:
# now that we can predict new values, we can determine the image from the training
#    set that best matches the last image
#    notice how it predicts an '8', which is the last number in the array
clf.predict(digits.data[-1:])

In [0]:

digits.target[:-1]

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

**In this exercise, I predicted the number corresponding to the last image in the **digits **dataset. **

2.  What was the most challenging part of what you did?

**The most challenging part was understanding what everything in the exercise meant. **

3.  What was the most interesting thing you learned?

**The most interesting thing I learned is that you can predict images from a training set.**

4.  What area would you like to explore with more time?

**I would like to explore more of image recognition.**




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.